In [2]:
# AOC Day 12

%load_ext autoreload
%autoreload 2

from aoc.utils import *
import math as math

In [68]:
def slice_region(region: set[tuple[int, int]]):
    min_r, min_c = math.inf, math.inf
    max_r, max_c = -math.inf, -math.inf
    for point in region:
        min_r = min(min_r, point[0])
        min_c = min(min_c, point[1])
        max_r = max(max_r, point[0])
        max_c = max(max_c, point[1])

    num_sides = 0
    side_started_left, side_started_right = False, False
    for c in range(min_c, max_c + 1):
        for r in range(min_r, max_r + 1):
            if (r, c) not in region:
                side_started_left, side_started_right = False, False
                continue

            left_is_border = (r, c - 1) not in region
            right_is_border = (r, c + 1) not in region
            if left_is_border and not side_started_left:
                num_sides += 1
                side_started_left = True
            if not left_is_border:
                side_started_left = False

            if right_is_border and not side_started_right:
                num_sides += 1
                side_started_right = True
            if not right_is_border:
                side_started_right = False
        side_started_left, side_started_right = False, False

    side_started_top, side_started_bottom = False, False
    for r in range(min_r, max_r + 1):
        for c in range(min_c, max_c + 1):
            if (r, c) not in region:
                side_started_top, side_started_bottom = False, False
                continue

            top_is_border = (r - 1, c) not in region
            bottom_is_border = (r + 1, c) not in region
            if top_is_border and not side_started_top:
                num_sides += 1
                side_started_top = True
            if not top_is_border:
                side_started_top = False

            if bottom_is_border and not side_started_bottom:
                num_sides += 1
                side_started_bottom = True
            if not bottom_is_border:
                side_started_bottom = False
        side_started_top, side_started_bottom = False, False
    
    return num_sides

class Farm:
    def __init__(self, file_address: str):
        self.map: dict[tuple[int, int], str] = {}
        self.searched: set[tuple[int, int]] = set()
        with open(file_address) as file:
            for i, row in enumerate(file.read().split("\n")):
                for j, c in enumerate(row):
                    self.map[(i, j)] = c
    
    def get(self, point: tuple[int, int]) -> int:
        return self.map.get(point, ".")
    
    def score_region(self, point: tuple[int, int]) -> int:
        if point in self.searched:
            return (0, 0)
        current_region_id = self.get(point)
        
        # Find the region
        point_added, region_set = True, set[tuple[int, int]]()
        region_set.add(point)
        while point_added:
            point_added = False
            new_region_set = region_set.copy()
            for point in region_set:
                for offset in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
                    new_point = (point[0] + offset[0], point[1] + offset[1])
                    if new_point in new_region_set:
                        continue
                    if self.get(new_point) == current_region_id:
                        new_region_set.add(new_point)
                        point_added = True
                        continue
                    if new_point in region_set:
                        continue
            region_set = new_region_set

        # Calculate the perimeter
        perimeter = 0
        for point in region_set:
            for offset in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
                new_point = (point[0] + offset[0], point[1] + offset[1])
                if new_point not in region_set:
                    perimeter += 1

        # Calculate the number of sides
        sides = slice_region(region_set)

        for point in region_set:
            self.searched.add(point)
        return len(region_set) * perimeter, len(region_set) * sides
    
    def iter_points(self):
        return self.map.keys()

(216, 120)


In [70]:

farm = Farm("input.txt")

part_1, part_2 = 0, 0
for point in farm.iter_points():
    score = farm.score_region(point)
    part_1 += score[0]
    part_2 += score[1]
print(part_1, part_2)

1471452 863366
